In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [4]:
q = "select * from matomo_traffic order by timestamp desc limit 500"
df = redshift_query_read(q, schema='production')

In [6]:
df.head(3)

,id,ip,visitorid,visitduration,actions,referrertype,referrername,devicetype,devicebrand,devicemodel,operatingsystemname,browser,url,timespent,pageviewposition,timestamp,org,form
0,4175929,174.198.73.124,520c211ce760e017,0,1,direct,None,Smartphone,Google,Pixel 6 Pro,Android,Chrome Mobile 107.0,https://secure.qgiv.com/for/t2tis-r/embed/mobi...,0,1,2022-11-19 23:59:55,0,0
1,4175928,50.38.138.251,718d456c1695f193,4,2,social,Facebook,Smartphone,Samsung,Galaxy A03s,Android,Facebook 392.2,https://secure.qgiv.com/for/fffcgd/embed?assoc...,0,2,2022-11-19 23:59:52,0,0
2,4175928,50.38.138.251,718d456c1695f193,4,2,social,Facebook,Smartphone,Samsung,Galaxy A03s,Android,Facebook 392.2,https://secure.qgiv.com/for/fffcgd/info/fbflag...,3,1,2022-11-19 23:59:49,0,0


In [2]:
q = "select id, org, path from form"
forms = redshift_query_read(q, schema='production')

In [3]:
print(len(forms))
forms.tail(3)

82577


,id,org,path
82574,1000088,447782,atr2023teamtest
82575,1000104,444088,2023jameswfoleyfreedomawards
82576,1000120,449399,foodpopup


In [ ]:
counter = 0
for _, f in forms.iterrows():
    q = '''update matomo_traffic set 
                form={}, org={} 
            where
                (url like '%/for/{}' or
                url like '%/for/{}/%' or
                url like '%/for/?key={}' or
                url like '%/for/?key={}&%' or
                url like '%/event/{}/%') and
                timestamp>='2022-12-01' and timestamp<='2022-12-31' '''.format(int(f['id']), int(f['org']), f['path'], f['path'], f['path'], f['path'], f['path'])
    redshift_query_write(q, schema='production')
    
    counter += 1
    if counter % 10000 == 0:
        print("done w/ {} forms".format(counter))